# Montée en ordre et conditionnement

In [ ]:
#
#    Notebook de cours MAP412 - Chapitre 3 - M. Massot 2020-2021 - Ecole polytechnique
#    ----------   
#    Montée en ordre, précision, conditionnement et stabilité 
#    
#    Auteurs : L. Séries et M. Massot - (C) 2021
#    

Dans cette partie, nous allons nous focaliser sur l'intégration de la fonction simple $f(x) = \cos(2x)$ sur l'intervalle $[0,1]$ dont le résultat exact est $0.5 \sin(2)$. Le but est ici de comprendre l'impact de la montée en ordre sur la précision des méthodes (Newton-Cotes, Clenshaw-Curtis et Gauss-Legendre) et d'identifier, potentiellement, le conditionnement du problème et la stabilité des algorithmes proposés.

In [ ]:
import numpy as np
import quadpy
import plotly.graph_objs as go
from scipy.integrate import newton_cotes
from scipy.special import roots_legendre

In [ ]:
def f(x):
    return np.cos(2*x)

res_exa = 0.5*np.sin(2)

## Formules de Newton-Cotes  

### Poids calculés avec scipy 

In [ ]:
s = np.arange(2, 41, 1)

err = np.zeros(s.size)

for i, si in enumerate(s):
    b, _ = newton_cotes(si-1, equal=1)
    b = b/(si-1)
    c = np.linspace(0, 1, si)
    res = np.sum(b * f(c))
    err[i] = np.abs(res - res_exa)
    
fig = go.Figure()
fig.add_trace(go.Scatter(x=s, y=err, mode='lines+markers'))
fig.update_xaxes(title="Nombre d'étages")
fig.update_yaxes(type="log", exponentformat = 'e', title="Erreur")
fig.show()

Sur ce graphique apparaît clairement une difficulté lorsque l'on monte en ordre. L'impact est tellement fort dans cette gamme de nombre de points de quadrature, en lien avec les études de l'interpolation menée dans le chapitre précédent, qu'il doit s'agir de la combinaison d'un mauvais conditionnement et d'un algorithme instable. Pour vérifier cela, on prend un autre angle d'attaque sur le calcul des poids en utilisant du calcul symbolique (on pourrait aussi utiliser Sagemath avec un calcul exact dans le corps des rationels) pour garantir que l'évaluation des poids est faite précisément.

### Poids calculés avec quadpy

In [ ]:
s = np.array([2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 16, 18, 20, 24, 28, 32, 36, 40])

err = np.zeros(s.size)
res_nc_quadpy = np.zeros(s.size)
b_nc_quadpy = []

for i, si in enumerate(s):
    #print(si, " ", end="")
    quad = quadpy.c1.newton_cotes_closed(si-1)
    b_nc_quadpy.append(0.5*quad.weights)
    c = np.linspace(0, 1, si)
    res_nc_quadpy[i] = np.sum(b_nc_quadpy[i] * f(c))
    err[i] = np.abs(res_nc_quadpy[i] - res_exa)

fig = go.Figure()
fig.add_trace(go.Scatter(x=s, y=err, mode='lines+markers'))
fig.update_xaxes(title="Nombre d'étages")
fig.update_yaxes(type="log", exponentformat = 'e', title="Erreur")
fig.show()

Le lecteur pourra constater que la dégradation de la précision est aussi présente, certainement due à un problème de conditionnement (points équi-distribués), mais beaucoup moins forte. Il s'agit donc d'être prudent et d'évaluer le conditionnement du problème. Il n'est pas difficile de faire le lien avec le conditionnement théorique étudié au chapitre précédent, mais on peut aussi en proposer une estimation numérique en format perturbatif.

### Perturbation des $f(c_i)$

In [ ]:
eps = 0.01

err = np.zeros(s.size)
err_pert = np.zeros(s.size)

for i, si in enumerate(s):
    c = np.linspace(0, 1, si)
    f_pert = f(c) + eps*(2*np.random.rand(c.size)-1)*f(c)
    res_pert = np.sum(b_nc_quadpy[i] * f_pert)
    err_pert[i] = np.abs(res_nc_quadpy[i] - res_pert)
    
fig = go.Figure()
fig.add_trace(go.Scatter(x=s, y=err_pert, mode='lines+markers'))
fig.update_xaxes(title="Nombre d'étages")
fig.update_yaxes(type="log", exponentformat = 'e', title="|Res. - Res. pert|")
fig.show()

On retrouve ici (la perturbation est aléatoire, donc esssayer plusieurs fois...) un conditionnement de l'ordre de $10.e7$ pour 40 étages, montrant que la perte de précision observée sur l'erreur "forward" est bien liée à une difficulté de conditionnement.

## Formule de Clenshaw-Curtis

In [ ]:
def xcheb(n):
    if n == 0:
        return np.array([])
    else:
        x = 0.5*np.cos( (2*np.arange(0,n)+1)*np.pi / (2*n) ) + 0.5
    return x

def coeffs_clenshawcurtis(n):
    x = xcheb(n+1) # n+1 Chebyshev nodes
    tab_k = np.arange(0, n+1, dtype='float')
    b = 1 / (tab_k + 1)
    M = np.vander(x, increasing=True)
    w = np.linalg.solve(np.transpose(M), b)
    return x, w    

In [ ]:
s = np.arange(2, 41, 1)

err = np.zeros(s.size)

for i, si in enumerate(s):
    c, b = coeffs_clenshawcurtis(si-1)
    res = np.sum(b * f(c))
    err[i] = np.abs(res - res_exa)
    if (err[i]==0): err[i]=1e-16    

fig = go.Figure()
fig.add_trace(go.Scatter(x=s, y=err, mode='lines+markers'))
fig.update_xaxes(title="Nombre d'étages")
fig.update_yaxes(type="log", exponentformat = 'e', title="Erreur")
fig.show()

Même si une estimation plus précise des poids d'intégration pourrait mener à une absence totale de perte de précision, on voit que le conditionnement est ici plus sympathique comme on pouvait le prévoir suite au Chapitre 2. 

### Perturbation des $f(c_i)$

In [ ]:
eps = 0.01

err_pert = np.zeros(s.size)

for i, si in enumerate(s):
    c, b = coeffs_clenshawcurtis(si-1)
    res = np.sum(b * f(c))
    err[i] = np.abs(res - res_exa)
    f_pert = f(c) + eps*(2*np.random.rand(c.size)-1)*f(c)
    res_pert = np.sum(b * f_pert)
    err_pert[i] = np.abs(res - res_pert)

fig = go.Figure()
fig.add_trace(go.Scatter(x=s, y=err_pert, mode='lines+markers'))
fig.update_xaxes(title="Nombre d'étages")
fig.update_yaxes(type="log", exponentformat = 'e', title="|Res. - Res. pert|")
fig.show()

On voit ici que le conditionnement reste très raisonnable mais le lecteur attentif s'apercevra que pour estimer le conditionnement numériquement d'un problème, il faut garantir que l'algorithme numérique est d'excellente stabilité, sous peine de ternir les conclusions. Une estimation plus précise des poids (calcul symbolique) permettrait ici de souligner l'excellent conditionnement du problème.

## Formule de Gauss

In [ ]:
s = np.arange(2, 41, 1)

err = np.zeros(s.size)

for i, si in enumerate(s):
    c, b = roots_legendre(si)
    c = 0.5*(c+1)
    b = 0.5*b
    res = np.sum(b * f(c))
    err[i] = np.abs(res - res_exa)
    if (err[i] == 0): err[i]=1e-16
    
fig = go.Figure()
fig.add_trace(go.Scatter(x=s, y=err, mode='lines+markers'))
fig.update_xaxes(title="Nombre d'étages")
fig.update_yaxes(type="log", exponentformat = 'e', title="Erreur")
fig.show()

### Formule perturbée 

In [ ]:
eps = 0.01

s = np.arange(2, 41, 1)

err = np.zeros(s.size)
err_pert = np.zeros(s.size)

for i, si in enumerate(s):
    c, b = roots_legendre(si)
    c = 0.5*(c+1)
    b = 0.5*b
    res = np.sum(b * f(c))
    f_pert = f(c) + eps*(2*np.random.rand(c.size)-1)*f(c)
    res_pert = np.sum(b * f_pert)
    err_pert[i] = np.abs(res - res_pert)
    
fig = go.Figure()
fig.add_trace(go.Scatter(x=s, y=err_pert, mode='lines+markers'))
fig.update_xaxes(title="Nombre d'étages")
fig.update_yaxes(type="log", exponentformat = 'e', title="|Res. - Res. pert|")
fig.show()

Le conditionnement est ici excellent et la perturbation n'est pas amplifiée quelque soit l'ordre de la méthode utilisée. 